In [17]:
:opt no-lint

# 19 Applying Structure
## 19.1 Applying Structure
The chapter features a few bites of real world Haskell code.
## 19.2 Monoid
### Templating content in Scotty
The `scotty` web framework’s “Hello, World” example uses `mconcat` to inject the parameter "word" into the HTML page returned.
### Concatenating connection parameters
The blog post Aditya Bhargava’s “Making A Website With Haskell,” shows using `foldr` and `Monoid` to concatenate connection parameters for connecting to a database.
### Concatenating key configurations
`xmonad` uses `Monoid` to merge custom keyboard mappings with default ones. In particular, it relies on the  `Monoid` instance of `-> b` when `b` is a `Monoid` (in this case `b` is `Data.Map`):

In [18]:
f = (++"a")
g = (++"b")
h = f <> g
h "c"

"cacb"

## 19.3 Functor
### Lifting over IO
The examples shows how to apply a pure function on a value wrapped into a `IO` context.
### Lifting over web app monads
The `snap` library wraps requests into a monadic context. `fmap` can be used to apply functions to results from an HTTP request.
## 19.4 Applicative
### hgrev
The example uses `Monoid` and `Applicative` to combine parsers of command line arguments.
### And now for something different
`Applicative` can be used for parallel application of the functions against an argument, followed by monoidally combine the values.
## 19.5 Monad
Because effectful programming requires `IO`, examples of `Monad` are everywhere. 
### Opening a network socket
In `haproxy-haskell` operations on sockets like creation, connections are composed with monadic bind.
### Binding over failure in initialization
Michael Xavier’s Seraph (a process monitor) has a typicial main entry point: the outermost `Monad` is `IO`, and the monad transformer `EitherT`, is used to bind over the possibility of failure during initialization.
## 19.6 An end-to-end example: URL shortener
The section presents a complete example, featuring:

- polymorphic literals, which rely on different instance of `IsString` class
- A Redis context which embeds `Web.Scotty` web requests (the `Monad` is `ActionM`)
- Lifting a `IO` action inside a Scotty `ActionM` in order to perform it

### Exercise
Ensure already existing short URLs are not overwritten.

In [19]:
{-# LANGUAGE OverloadedStrings #-}

import Control.Monad (replicateM)
import Control.Monad.IO.Class (liftIO)
import qualified Data.ByteString.Char8 as BC
import Data.Text.Encoding (decodeUtf8, encodeUtf8)
import qualified Data.Text.Lazy as TL
import qualified Database.Redis as R
import Network.URI (URI, parseURI)
import qualified System.Random as SR
import Web.Scotty

alphaNum :: String
alphaNum = ['A'..'Z'] ++ ['0'..'9']

randomElement :: String -> IO Char
randomElement xs = do
  let maxIndex :: Int
      maxIndex = length xs - 1
  randomDigit <- SR.randomRIO (0, maxIndex)
  return (xs !! randomDigit)

shortyGen :: IO [Char]
shortyGen = replicateM 7 (randomElement alphaNum)

saveURI :: R.Connection -> BC.ByteString -> BC.ByteString -> IO (Either R.Reply R.Status)
saveURI conn shortURI uri = R.runRedis conn $ R.set shortURI uri

getURI :: R.Connection -> BC.ByteString -> IO (Either R.Reply (Maybe BC.ByteString))
getURI conn shortURI = R.runRedis conn $ R.get shortURI

linkShorty :: String -> String
linkShorty shorty = concat [ "<a href=\"", shorty, "\">Copy and paste your short URL</a>" ]

shortyCreated :: Show a => a -> String -> TL.Text
shortyCreated resp shawty = TL.concat [ TL.pack (show resp), " shorty is: ", TL.pack (linkShorty shawty) ]

shortyAintUri :: TL.Text -> TL.Text
shortyAintUri uri = TL.concat [uri, " wasn't a url, did you forget http://?"]

shortyFound :: TL.Text -> TL.Text
shortyFound tbs = TL.concat ["<a href=\"", tbs, "\">", tbs, "</a>"]

shortyDuplicated :: TL.Text -> TL.Text
shortyDuplicated shorty = TL.concat [ "Shorty ", shorty, " already in use."]

app :: R.Connection -> ScottyM ()
app rConn = do
  get "/" $ do
    uri <- param "uri"
    let parsedUri :: Maybe URI
        parsedUri = parseURI (TL.unpack uri)
    case parsedUri of
      Nothing -> text (shortyAintUri uri)
      Just _  -> do
        shawty <- liftIO shortyGen
        let shorty = BC.pack shawty
        checkDuplicated <- liftIO $ getURI rConn shorty
        case checkDuplicated of
          Left reply -> text $ TL.pack $ show reply
          Right mbBS -> case mbBS of
            Just bs -> text $ shortyDuplicated $ TL.pack shawty
            Nothing -> do
              let uri' = encodeUtf8 $ TL.toStrict uri
              resp <- liftIO $ saveURI rConn shorty uri'
              html $ shortyCreated resp shawty
  get "/:short" $ do
    short <- param "short"
    uri <- liftIO (getURI rConn short)
    case uri of
      Left reply -> text (TL.pack (show reply))
      Right mbBS -> case mbBS of
        Nothing -> text "uri not found"
        Just bs -> html (shortyFound tbs)
           where tbs :: TL.Text
                 tbs = TL.fromStrict (decodeUtf8 bs)

main :: IO ()
main = do
  rConn <- R.connect R.defaultConnectInfo
  scotty 3000 (app rConn)

## 19.7 That’s a wrap!
The goal of the chapter was to show of how Haskellers use the type classes discussed in previous chapters in real code.